In [1]:
!pip install wheel idstools pandas numpy

In [2]:
import pandas as pd
import numpy as np

from idstools import rule

import glob
import json
import os
import re

# ET Open

Download ET Open ruleset. 
```
wget https://rules.emergingthreats.net/open/suricata-6.0.1/emerging.rules.tar.gz
```

And unpack.

```
mkdir /tmp/etopen
tar -xzf emerging.rules.tar.gz -C /tmp/etopen
```

In [3]:
!wget -q -O /tmp/etopen.tgz https://rules.emergingthreats.net/open/suricata-6.0.1/emerging.rules.tar.gz

In [4]:
!mkdir -p /tmp/etopen
!tar -xzf /tmp/etopen.tgz -C /tmp/etopen

Note that this folder in `tmp` must be synced with following `glob` code which constructs a python list of all rule files.

In [5]:
RULES_LIST_ET_OPEN = glob.glob("/tmp/etopen/rules/*.rules")

Then use python code to get a organized list of rule files.

In [6]:
sorted(RULES_LIST_ET_OPEN)

['/tmp/etopen/rules/3coresec.rules',
 '/tmp/etopen/rules/botcc.portgrouped.rules',
 '/tmp/etopen/rules/botcc.rules',
 '/tmp/etopen/rules/ciarmy.rules',
 '/tmp/etopen/rules/compromised.rules',
 '/tmp/etopen/rules/drop.rules',
 '/tmp/etopen/rules/dshield.rules',
 '/tmp/etopen/rules/emerging-activex.rules',
 '/tmp/etopen/rules/emerging-adware_pup.rules',
 '/tmp/etopen/rules/emerging-attack_response.rules',
 '/tmp/etopen/rules/emerging-chat.rules',
 '/tmp/etopen/rules/emerging-coinminer.rules',
 '/tmp/etopen/rules/emerging-current_events.rules',
 '/tmp/etopen/rules/emerging-deleted.rules',
 '/tmp/etopen/rules/emerging-dns.rules',
 '/tmp/etopen/rules/emerging-dos.rules',
 '/tmp/etopen/rules/emerging-exploit.rules',
 '/tmp/etopen/rules/emerging-exploit_kit.rules',
 '/tmp/etopen/rules/emerging-ftp.rules',
 '/tmp/etopen/rules/emerging-games.rules',
 '/tmp/etopen/rules/emerging-hunting.rules',
 '/tmp/etopen/rules/emerging-icmp.rules',
 '/tmp/etopen/rules/emerging-icmp_info.rules',
 '/tmp/etopen

And parse each rule file with `idstools`, and construct a python dictionary where keys are rule files and values are list of parsed rules.

In [7]:
%time PARSED_ET_OPEN = {k: rule.parse_file(k) for k in RULES_LIST_ET_OPEN}

CPU times: user 1.66 s, sys: 131 ms, total: 1.79 s
Wall time: 1.79 s


Consider the following parsed rule. Notice how much information can be extracted from it. And reader should already be familiar with sequential option list.

In [8]:
print(
    json.dumps(
        PARSED_ET_OPEN["/tmp/etopen/rules/emerging-malware.rules"][0], 
        indent=2
    )
)

{
  "enabled": false,
  "action": "alert",
  "direction": "->",
  "group": null,
  "gid": 1,
  "sid": 2009172,
  "rev": 2,
  "msg": "ET MALWARE Psyb0t joining an IRC Channel",
  "flowbits": [
    "isset,is_proto_irc"
  ],
  "metadata": [
    "created_at 2010_07_30",
    "updated_at 2010_07_30"
  ],
  "references": [
    "url,www.adam.com.au/bogaurd/PSYB0T.pdf",
    "url,doc.emergingthreats.net/2009172"
  ],
  "classtype": "trojan-activity",
  "priority": 0,
  "options": [
    {
      "name": "msg",
      "value": "\"ET MALWARE Psyb0t joining an IRC Channel\""
    },
    {
      "name": "flow",
      "value": "established,to_server"
    },
    {
      "name": "flowbits",
      "value": "isset,is_proto_irc"
    },
    {
      "name": "content",
      "value": "\"JOIN #mipsel\""
    },
    {
      "name": "reference",
      "value": "url,www.adam.com.au/bogaurd/PSYB0T.pdf"
    },
    {
      "name": "reference",
      "value": "url,doc.emergingthreats.net/2009172"
    },
    {
      "name

## High level view

Traditional data structures can be difficult for human eyes to grasp. On small scale they are fine, but things become complex if you consider that ET Open contains over 31 **thousand** rules. However, aggregations presented in row-column format can help us out here.

For that, we can use `pandas` scientific package which implements **data frames** in python. Great for data wrangling and exploration. Following block creates a new pandas data frame, and initializes columns of counters per rule file. For now, we're just interested in `total number of rules`, `number of enabled rules` and `number of disabled rules`.

In [9]:
DF_HIGH_LEVEL = pd.DataFrame()
DF_HIGH_LEVEL["file"] = list(PARSED_ET_OPEN.keys())
DF_HIGH_LEVEL["rules_total_count"] = list([len(v) for v in PARSED_ET_OPEN.values()])
DF_HIGH_LEVEL["rules_disabled_count"] = list([len([item for item in v if not item.enabled]) for v in PARSED_ET_OPEN.values()])
DF_HIGH_LEVEL["rules_enabled_count"] = list([len([item for item in v if item.enabled]) for v in PARSED_ET_OPEN.values()])

Then present the dataframe sorted by the number of active rules per file.

In [10]:
DF_HIGH_LEVEL.sort_values(by=["rules_enabled_count"], ascending=False)

,file,rules_total_count,rules_disabled_count,rules_enabled_count
10,/tmp/etopen/rules/emerging-malware.rules,12180,2545,9635
26,/tmp/etopen/rules/emerging-web_specific_apps.r...,5600,749,4851
21,/tmp/etopen/rules/emerging-phishing.rules,2249,106,2143
18,/tmp/etopen/rules/emerging-exploit.rules,1773,320,1453
30,/tmp/etopen/rules/emerging-info.rules,1283,60,1223
46,/tmp/etopen/rules/emerging-hunting.rules,1051,38,1013
16,/tmp/etopen/rules/tor.rules,987,0,987
35,/tmp/etopen/rules/emerging-policy.rules,1209,327,882
41,/tmp/etopen/rules/emerging-mobile_malware.rules,913,78,835
37,/tmp/etopen/rules/emerging-web_server.rules,796,147,649


Each column of counters is a vector that can be summed up for total counts.

In [11]:
print("Enabled: {} Disabled: {} Total: {}".format(
    DF_HIGH_LEVEL.rules_enabled_count.sum(),
    DF_HIGH_LEVEL.rules_disabled_count.sum(),
    DF_HIGH_LEVEL.rules_total_count.sum(),
))

Enabled: 28166 Disabled: 10964 Total: 39130


## Dig into specific rule files and threats

Okay, now let's try to get information about some rules themselves.

Before getting started, `idstools` parses some information that is not terribly useful (like `action`, `direction`) while leaving other more useful data pieces unparsed. Looking specifically the `header` field for `protocol`, `src_net` and `dest_net`. Following helper function can parse that information.

In [12]:
def extract_header(header: str) -> dict:
    split = header.split()
    return {
        "proto": split[1],
        "src_net": split[2],
        "src_port": split[3],
        "dest_net": split[5],
        "dest_port": split[6]
    }

Then build a list of all rules while adding cleaned up filename and that `header` information to dictionary.

In [13]:
ALL_ET_OPEN_RULES = []

for filename, rules in PARSED_ET_OPEN.items():
    for r in rules:
        r["file"] = os.path.basename(filename)
        r = {**r, **extract_header(r.get("header"))}
        ALL_ET_OPEN_RULES.append(r)

And rather than attempting to inspect 31k element list, we'll turn the whole thing into a dataframe.

In [14]:
DF_ET_OPEN_ALL = pd.DataFrame(ALL_ET_OPEN_RULES)

Filter for only enabled rules. Rules are always commented for a reason!
* false positives;
* bad performance;
* just out of date and irrelevant;

In [15]:
DF_ET_OPEN_ALL = DF_ET_OPEN_ALL.loc[DF_ET_OPEN_ALL.enabled == True]

And get a quick peek of ruleset. Just to see what we can work on. Clearly we need to do more filtering and a proper selection of columns. All those *sticky buffer* and *content modifier* columns are totally useless. That's because they always apply to `content` keyword and have no values themselves. Thus, all those vectors are empty.

In [16]:
DF_ET_OPEN_ALL.head(5)

,enabled,action,direction,group,gid,sid,rev,msg,flowbits,metadata,...,icmp_seq,ja3_hash,ssh.software,http_referer,ja3s.hash,tls.fingerprint,ja3.hash,ja3.string,ftpbounce,sameip
1,True,alert,->,None,1,2102003,9,GPL SQL Slammer Worm propagation attempt,[],"[created_at 2010_09_23, updated_at 2010_09_23]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,alert,->,None,1,2101775,4,GPL SQL MYSQL root login attempt,[],"[created_at 2010_09_23, updated_at 2010_09_23]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,alert,->,None,1,2101776,4,GPL SQL MYSQL show databases attempt,[],"[created_at 2010_09_23, updated_at 2010_09_23]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,True,alert,->,None,1,2101674,6,GPL SQL connect_data remote version detection ...,[],"[created_at 2010_09_23, updated_at 2010_09_23]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,True,alert,->,None,1,2103152,4,GPL SQL sa brute force failed login attempt,[],"[created_at 2010_09_23, updated_at 2010_09_23]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So, we'll build a more consise dataframe. with only those columns we are about. List is not exhaustive and just my selection. **Decide what is relevant to you!**

In [17]:
CORE_COLS = ["proto", "src_net", "dest_net", "sid", "rev", "msg", "file", "flowbits", "metadata", "references", "flow", "raw"]

In [18]:
DF_ET_OPEN_CONSISE = DF_ET_OPEN_ALL.loc[:,  CORE_COLS]

Notice that our dataframe peek was truncated. This is to avoid exploding your browser, as dataframes can be very big. Following optins can disable that to reveal more information. **But use them with care, make sure you don't call 31k row printout into your browser!**

In [19]:
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.min_rows', None)
pd.set_option('display.width', None)

Some rule categories are small and can be shown as-is. Rather than creating separate data structures, we'll go data science way and keep everything in one dataframe. Remember, we are exploring, so we never know where that exploration will lead. Better to keep everything at arms reach and just filter if needed. Rely on intermediete data before reaching your goal.

So, to see into `emerging-worm` category, we can simply filter for that file name. Furthermore, we can sort values to make the information easier to grasp. Sorting by rule directionality is already a good trick to visually group rules.

In [20]:
DF_ET_OPEN_CONSISE \
    .loc[DF_ET_OPEN_CONSISE.file.str.contains("emerging-worm.rules")] \
    .sort_values(by=["src_net", "dest_net"])

,proto,src_net,dest_net,sid,rev,msg,file,flowbits,metadata,references,flow,raw
425,udp,$HOME_NET,$EXTERNAL_NET,2102004,8,GPL WORM Slammer Worm propagation attempt OUTBOUND,emerging-worm.rules,[],"[created_at 2010_09_23, updated_at 2010_09_23]","[bugtraq,5310, bugtraq,5311, cve,2002-0649, nessus,11214, url,vil.nai.com/vil/content/v_99992.htm]",NaN,"alert udp $HOME_NET any -> $EXTERNAL_NET 1434 (msg:""GPL WORM Slammer Worm propagation attempt OUTBOUND""; content:""|04|""; depth:1; content:""|81 F1 03 01 04 9B 81 F1|""; content:""sock""; content:""send""; reference:bugtraq,5310; reference:bugtraq,5311; reference:cve,2002-0649; reference:nessus,11214; reference:url,vil.nai.com/vil/content/v_99992.htm; classtype:misc-attack; sid:2102004; rev:8; metadata:created_at 2010_09_23, updated_at 2010_09_23;)"
428,tcp,$HOME_NET,$EXTERNAL_NET,2017404,3,ET WORM W32/Njw0rm CnC Beacon,emerging-worm.rules,[],"[attack_target Client_Endpoint, created_at 2013_08_31, deployment Perimeter, former_category WORM, signature_severity Major, tag c2, updated_at 2013_08_31, mitre_tactic_id TA0011, mitre_tactic_name Command_And_Control, mitre_technique_id T1041, mitre_technique_name Exfiltration_Over_C2_Channel]","[url,www.fireeye.com/blog/technical/malware-research/2013/08/njw0rm-brother-from-the-same-mother.html, md5,4c60493b14c666c56db163203e819272, md5,b0e1d20accd9a2ed29cdacb803e4a89d]","established,to_server","alert tcp $HOME_NET any -> $EXTERNAL_NET any (msg:""ET WORM W32/Njw0rm CnC Beacon""; flow:established,to_server; content:""lv0njxq80""; depth:9; content:""njxq80""; distance:0; reference:url,www.fireeye.com/blog/technical/malware-research/2013/08/njw0rm-brother-from-the-same-mother.html; reference:md5,4c60493b14c666c56db163203e819272; reference:md5,b0e1d20accd9a2ed29cdacb803e4a89d; classtype:command-and-control; sid:2017404; rev:3; metadata:attack_target Client_Endpoint, created_at 2013_08_31, deployment Perimeter, former_category WORM, signature_severity Major, tag c2, updated_at 2013_08_31, mitre_tactic_id TA0011, mitre_tactic_name Command_And_Control, mitre_technique_id T1041, mitre_technique_name Exfiltration_Over_C2_Channel;)"
430,http,$HOME_NET,$EXTERNAL_NET,2014402,3,ET WORM W32/Rimecud wg.txt Checkin,emerging-worm.rules,[],"[created_at 2012_03_20, updated_at 2020_04_21]","[md5,a89f7289d5cce821a194542e90026082, md5,fd56ce176889d4fbe588760a1da6462b, url,www.microsoft.com/security/portal/Threat/Encyclopedia/Entry.aspx?Name=Worm%3AWin32%2FRimecud]","established,to_server","alert http $HOME_NET any -> $EXTERNAL_NET any (msg:""ET WORM W32/Rimecud wg.txt Checkin""; flow:established,to_server; http.uri; content:""/wg.txt""; reference:md5,a89f7289d5cce821a194542e90026082; reference:md5,fd56ce176889d4fbe588760a1da6462b; reference:url,www.microsoft.com/security/portal/Threat/Encyclopedia/Entry.aspx?Name=Worm%3AWin32%2FRimecud; classtype:trojan-activity; sid:2014402; rev:3; metadata:created_at 2012_03_20, updated_at 2020_04_21;)"
433,http,$HOME_NET,$EXTERNAL_NET,2012201,5,ET WORM Possible Worm Sohanad.Z or Other Infection Request for setting.nql,emerging-worm.rules,[],"[created_at 2011_01_17, updated_at 2020_08_04]","[md5,a70aad8f27957702febfa162556dc5b5]","established,to_server","alert http $HOME_NET any -> $EXTERNAL_NET any (msg:""ET WORM Possible Worm Sohanad.Z or Other Infection Request for setting.nql""; flow:established,to_server; http.uri; content:""/setting.nql""; nocase; reference:md5,a70aad8f27957702febfa162556dc5b5; classtype:trojan-activity; sid:2012201; rev:5; metadata:created_at 2011_01_17, updated_at 2020_08_04;)"
435,http,$HOME_NET,$EXTERNAL_NET,2008020,6,ET WORM Win32.Socks.s HTTP Post Checkin,emerging-worm.rules,[],"[created_at 2010_07_30, updated_at 2020_08_18]","[url,doc.emergingthreats.net/2008020]","established,to_server","alert http $HOME_NET any -> $EXTERNAL_NET any (msg:""ET WORM Win32.Socks.s HTTP Post Checkin""; flow:established,to_server; http.method; content:""POST""; http.uri; content:"".php""; http.request_body; content:"

**PS! Jupyter is a data science tool, and thus caters to that audience. This can lead to silly things like formating rule header like mathematical formula**.

However, really good stuff is in `malware` and `mobile_malware` categories. And those are big. Too big to explore with full dumps. So, let's limit the scope only to a *recent hotness*.

In [21]:
RULES_SUNBURST = DF_ET_OPEN_CONSISE \
    .loc[DF_ET_OPEN_CONSISE.msg.str.contains("SUNBURST", re.IGNORECASE)] \
    .sort_values(by=["proto", "src_net", "dest_net", "msg"]) \
    .drop(columns=["flowbits", "raw", "metadata", "flow"]) \
    .explode("references")

This is a bit more involved, but in many ways is similar to a database query.
* First, we locate all rules containing `SUNBURST` keyword. Sometimes this information is in `tag` or `metadata`, but dont count on it. And it's not very consistent.
* Then we sort values to make the frame visually easier to explore. Pandas even let's us sort by multiple values. That's why I wanted to parse `proto`, `src_net` and `dest_net` from the rule header! With those fields, we get a much better organized view.
* Then drop some columns (from view) that are just noise:
  * `flowbits` are not really that relevant for current explorations, rule content should be listed separately anyway
  * likewise `raw` rule just makes dataframe as a whole more difficult to assess, but it can always be added back if we need to check the content!
  * `metadata` does not hold much useful information and is a list, which again makes frame messy
  * `flow` is a bit redundant with sorted `src_net` and `dest_net` view. Good info, but we only have limited screen real-estate
 * Finally, `references` holds lists, but we can use `explode()` method to unpack each reference to separate row. **This duplicates other rule row elements!** But not a big deal for this case.

In [22]:
RULES_SUNBURST

,proto,src_net,dest_net,sid,rev,msg,file,references
9285,dns,$HOME_NET,any,2031392,1,ET MALWARE Dark Halo/SUNBURST Related DNS Lookup to globalnetworkissues .com,emerging-malware.rules,"url,www.volexity.com/blog/2020/12/14/dark-halo-leverages-solarwinds-compromise-to-breach-organizations/"
9284,dns,$HOME_NET,any,2031391,1,ET MALWARE Dark Halo/SUNBURST Related DNS Lookup to kubecloud .com,emerging-malware.rules,"url,www.volexity.com/blog/2020/12/14/dark-halo-leverages-solarwinds-compromise-to-breach-organizations/"
9282,dns,$HOME_NET,any,2031389,1,ET MALWARE Dark Halo/SUNBURST Related DNS Lookup to lcomputers .com,emerging-malware.rules,"url,www.volexity.com/blog/2020/12/14/dark-halo-leverages-solarwinds-compromise-to-breach-organizations/"
9283,dns,$HOME_NET,any,2031390,1,ET MALWARE Dark Halo/SUNBURST Related DNS Lookup to seobundlekit .com,emerging-malware.rules,"url,www.volexity.com/blog/2020/12/14/dark-halo-leverages-solarwinds-compromise-to-breach-organizations/"
9281,dns,$HOME_NET,any,2031387,1,ET MALWARE Dark Halo/SUNBURST Related DNS Lookup to solartrackingsystem .net,emerging-malware.rules,"url,www.volexity.com/blog/2020/12/14/dark-halo-leverages-solarwinds-compromise-to-breach-organizations/"
...,...,...,...,...,...,...,...,...
9272,tls,$HOME_NET,$EXTERNAL_NET,2031363,1,ET MALWARE [Fireeye] Observed Backdoor.SUNBURST CnC Domain (incomeudpate .com in TLS SNI),emerging-malware.rules,"url,www.fireeye.com/blog/threat-research/2020/12/evasive-attacker-leverages-solarwinds-supply-chain-compromises-with-sunburst-backdoor.html"
9273,tls,$HOME_NET,$EXTERNAL_NET,2031364,1,ET MALWARE [Fireeye] Observed Backdoor.SUNBURST CnC Domain (panhardware .com in TLS SNI),emerging-malware.rules,"url,www.fireeye.com/blog/threat-research/2020/12/evasive-attacker-leverages-solarwinds-supply-chain-compromises-with-sunburst-backdoor.html"
9271,tls,$HOME_NET,$EXTERNAL_NET,2031362,1,ET MALWARE [Fireeye] Observed Backdoor.SUNBURST CnC Domain (thedoccloud .com in TLS SNI),emerging-malware.rules,"url,www.fireeye.com/blog/threat-research/2020/12/evasive-attacker-leverages-solarwinds-supply-chain-compromises-with-sunburst-backdoor.html"
9277,tls,$HOME_NET,$EXTERNAL_NET,2031368,1,ET MALWARE [Fireeye] Observed Backdoor.SUNBURST CnC Domain (websitetheme .com in TLS SNI),emerging-malware.rules,"url,www.fireeye.com/blog/threat-research/2020/12/evasive-attacker-leverages-solarwinds-supply-chain-compromises-with-sunburst-backdoor.html"


Same exploration can be repeated for other relevant threats. For example, I bet many students are interested in `Cobalt Strike` rules.

In [23]:
RULES_COBALT_STRIKE = DF_ET_OPEN_CONSISE \
    .loc[DF_ET_OPEN_CONSISE \
    .msg.str.contains("Cobalt Strike|CobaltStrike", re.IGNORECASE)] \
    .drop(columns=["metadata", "flowbits"]) \
    .explode("references") \
    .sort_values(by=["msg"]) \
    .drop(columns=["raw"])

In [24]:
RULES_COBALT_STRIKE.head()

,proto,src_net,dest_net,sid,rev,msg,file,references,flow
37110,tls,$EXTERNAL_NET,$HOME_NET,2023629,5,ET HUNTING Suspicious Empty SSL Certificate - Observed in Cobalt Strike,emerging-hunting.rules,NaN,"from_server,established"
36559,tls,$EXTERNAL_NET,$HOME_NET,2033396,1,ET JA3 HASH - Possible Cobalt Strike Server,emerging-ja3.rules,"url,thedfirreport.com/2021/07/19/icedid-and-cobalt-strike-vs-antivirus/",NaN
36558,tls,$EXTERNAL_NET,$HOME_NET,2033395,1,ET JA3 HASH - Possible Cobalt Strike Server,emerging-ja3.rules,"url,thedfirreport.com/2021/07/19/icedid-and-cobalt-strike-vs-antivirus/",NaN
36557,tls,$EXTERNAL_NET,$HOME_NET,2033157,1,ET JA3 HASH - Possible Cobalt Strike Server,emerging-ja3.rules,"url,thedfirreport.com/2021/06/20/from-word-to-lateral-movement-in-1-hour/",NaN
36560,tls,$EXTERNAL_NET,$HOME_NET,2028832,3,ET JA3 Hash - Suspected Cobalt Strike Malleable C2 (ja3s) M1,emerging-ja3.rules,NaN,"established,from_server"


Here we can see that many rules have multiple references. And, on that note, rules can hold a lot of interesting reading materials! How about we build a reading list.

In [25]:
TEXT = "\n".join(sorted(
    list(
        RULES_COBALT_STRIKE \
            .loc[RULES_COBALT_STRIKE.fillna("NA") \
                                    .references.str.contains("^url")] \
            .references.unique()
    )
))

In [26]:
print(TEXT)

url,attack.mitre.org/groups/G0080/
url,blog.cobaltstrike.com/2015/10/07/named-pipe-pivoting/
url,blog.malwarebytes.com/threat-analysis/2020/06/multi-stage-apt-attack-drops-cobalt-strike-using-malleable-c2-feature
url,blog.malwarebytes.com/threat-intelligence/2022/03/new-spear-phishing-campaign-targets-russian-dissidents/
url,blog.reversinglabs.com/blog/threat-analysis-follina-exploit-powers-live-off-the-land-attacks
url,blog.talosintelligence.com/2020/06/indigodrop-maldocs-cobalt-strike.html
url,blog.talosintelligence.com/2021/11/attackers-use-domain-fronting-technique.html
url,blog.talosintelligence.com/2022/05/mustang-panda-targets-europe.html
url,cert.gov.ua/article/37704
url,cert.gov.ua/article/38155
url,cert.gov.ua/article/39708
url,cert.gov.ua/article/40559
url,cert.gov.ua/article/619229
url,cert.gov.ua/article/703548
url,cisa.gov/uscert/ncas/alerts/aa21-265a
url,documents.trendmicro.com/assets/white_papers/wp-earth-baku-an-apt-group-targeting-indo-pacific-countries.pdf
url,firee

But note that many links might be dead.

In [27]:
RULES_PURPLE_FOX = DF_ET_OPEN_CONSISE \
    .loc[DF_ET_OPEN_CONSISE \
    .msg.str.contains("PurpleFox", re.IGNORECASE)] \
    .drop(columns=["metadata", "flowbits"]) \
    .explode("references") \
    .sort_values(by=["msg"]) \
    .drop(columns=["raw"])

In [28]:
RULES_PURPLE_FOX

,proto,src_net,dest_net,sid,rev,msg,file,references,flow
16297,http,$HOME_NET,$EXTERNAL_NET,2028978,3,ET EXPLOIT_KIT Possible PurpleFox EK Framework Flash GET Request,emerging-exploit_kit.rules,NaN,"established,to_server"
16296,http,$HOME_NET,$EXTERNAL_NET,2028977,3,ET EXPLOIT_KIT Possible PurpleFox EK Framework Flash HEAD Request,emerging-exploit_kit.rules,NaN,"established,to_server"
16113,http,$EXTERNAL_NET,$HOME_NET,2028974,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework Landing,emerging-exploit_kit.rules,NaN,"established,to_client"
16114,http,$EXTERNAL_NET,$HOME_NET,2028975,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework Landing - Various Exploits,emerging-exploit_kit.rules,NaN,"established,to_client"
16295,http,$EXTERNAL_NET,$HOME_NET,2028976,3,ET EXPLOIT_KIT Possible PurpleFox EK Framework Payload,emerging-exploit_kit.rules,NaN,"established,to_client"
16293,http,$EXTERNAL_NET,$HOME_NET,2028982,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework Payload,emerging-exploit_kit.rules,NaN,"established,to_client"
16298,http,$EXTERNAL_NET,$HOME_NET,2028981,3,ET EXPLOIT_KIT Possible PurpleFox EK Framework Payload,emerging-exploit_kit.rules,NaN,"established,to_client"
16292,http,$HOME_NET,$EXTERNAL_NET,2028980,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework URI Struct Flash Request,emerging-exploit_kit.rules,NaN,"established,to_server"
16334,http,$HOME_NET,$EXTERNAL_NET,2031466,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework URI Struct Jpg Request,emerging-exploit_kit.rules,NaN,"established,to_server"
16291,http,$HOME_NET,$EXTERNAL_NET,2028979,2,ET EXPLOIT_KIT Possible PurpleFox EK Framework URI Struct Landing Request,emerging-exploit_kit.rules,NaN,"established,to_server"


In [29]:
RULES_EMOTET = DF_ET_OPEN_CONSISE \
    .loc[DF_ET_OPEN_CONSISE \
    .msg.str.contains("Emotet", re.IGNORECASE)] \
    .drop(columns=["metadata", "flowbits"]) \
    .explode("references") \
    .sort_values(by=["msg"]) \
    .drop(columns=["raw"])

In [30]:
RULES_EMOTET

,proto,src_net,dest_net,sid,rev,msg,file,references,flow
6031,tls,$EXTERNAL_NET,$HOME_NET,2035063,2,ET MALWARE Emotet Certificate Observed M2,emerging-malware.rules,"md5,8430d8cf1b1edd6c49092a7dd6412a8a","established,to_client"
5106,http,$HOME_NET,$EXTERNAL_NET,2019693,6,ET MALWARE Emotet Checkin,emerging-malware.rules,"md5,3083b68cb5c2a345972a5f79e735c7b9","established,to_server"
9234,http,$HOME_NET,$EXTERNAL_NET,2035075,6,ET MALWARE Emotet CnC Beacon,emerging-malware.rules,"md5,518d189f8922280c81ab123604076dfd","established,to_server"
7969,http,$HOME_NET,$EXTERNAL_NET,2019704,4,ET MALWARE Emotet CnC Beacon,emerging-malware.rules,"md5,e24831e3f808116b30d85731c545e3ee","established,to_server"
3368,http,$EXTERNAL_NET,$HOME_NET,2035042,2,ET MALWARE Emotet Post Drop C2 Comms,emerging-malware.rules,NaN,"established,from_server"
8943,http,$EXTERNAL_NET,$HOME_NET,2035049,4,ET MALWARE Emotet Post Drop C2 Comms M2,emerging-malware.rules,"md5,dacdcd451204265ad6f44ef99db1f371","established,from_server"
4246,http,$HOME_NET,$EXTERNAL_NET,2029398,2,ET MALWARE Emotet Wifi Bruter Module Checkin,emerging-malware.rules,"url,www.binarydefense.com/emotet-evolves-with-new-wi-fi-spreader","established,to_server"
5186,http,$HOME_NET,$EXTERNAL_NET,2020900,4,ET MALWARE Emotet v2 Exfiltrating Outlook information,emerging-malware.rules,"url,securelist.com/analysis/69560/the-banking-trojan-emotet-detailed-analysis/","established,to_server"
6185,http,$HOME_NET,$EXTERNAL_NET,2035060,3,ET MALWARE Evil PDF Retrieving Emotet Payload,emerging-malware.rules,"md5,136dca58d0a0802c7abfce8dce4b7526","established,to_server"
6046,tls,$EXTERNAL_NET,$HOME_NET,2035051,3,ET MALWARE IcedID/Emotet Certificate Observed M1,emerging-malware.rules,"md5,8430d8cf1b1edd6c49092a7dd6412a8a","established,to_client"


## Interactive widgets

* https://ipywidgets.readthedocs.io/en/latest/

Not all data exploration must be done with pure code any more. Widgets are a great way to expose any data user is interested in.

In [31]:
! pip install ipywidgets

In [32]:
import re

In [33]:
def show_rules(limit: int, msg: str, columns: tuple, sort: tuple):
    pd.set_option('display.max_rows', limit)
    pd.set_option('display.min_rows', limit)
    return (
        DF_ET_OPEN_ALL[list(columns)]
        .loc[DF_ET_OPEN_ALL.msg.str.contains(msg, flags=re.IGNORECASE)]
        .sort_values(by=[c for c in list(sort) if c in list(columns)])
    )

In [34]:
import ipywidgets as widgets

In [35]:
widgets.interact(
    show_rules, 
    msg="", 
    limit=widgets.IntSlider(min=10, max=100),
    columns=widgets.SelectMultiple(
        options=list(DF_ET_OPEN_ALL.columns.values),
        value=CORE_COLS
    ),
    sort=widgets.SelectMultiple(
        options=list(DF_ET_OPEN_ALL.columns.values),
    )
)

interactive(children=(IntSlider(value=10, description='limit', min=10), Text(value='', description='msg'), Sel…

<function __main__.show_rules(limit: int, msg: str, columns: tuple, sort: tuple)>